#### 说明
* 功能：针对分类模型训练，生成离线采样样本及其它评测结果
    * 分割方法输出概率[0,1]范围映射到[0, 10000], 采样实际保留病灶阈值0.3

In [16]:
import os
import cv2
import shutil
from glob import glob
import numpy as np
import nibabel as nib
from skimage import measure
from joblib import Parallel,delayed
from multiprocessing import cpu_count

import torch
import torch.nn.functional as F

from sklearn.metrics import roc_auc_score

In [17]:
#分3个目录，分别处理训练、验证、测试3个分组的处理结果
data_group = {
    'train': './src/part_train.txt',
    'valid': './src/part_valid.txt',
    'test': './src/part_test.txt',
}
ground_truth_mask_root_map = {
    'train': './src/mask',
    'valid': './src/mask',
    'test': './src/mask',
} # 标注结果，实例化病灶，注意不同类别对应的病灶含义

# lesion_info_path_list = [
#     # './src/lesion_level_group_infos.json'
# ]

image_root_map = {
    'train': './src/image',
    'valid': './src/image',
    'test': './src/image',

}

segmentation_prob_root_map = {
    'train': './src/prob', 
    'valid': './src/prob', 
    'test': './src/prob', 
} # 分割概率值结果, _prob.nii.gz


save_root = './P0.3_wwwl_800_300_lesion_match_s3s36'


# ww_wl = 1200, 400
ww_wl = 800, 300 # 工程化代码
shape_norm = (36,)*3 # 归一化大小
used_ctx_scale = 3

In [18]:
# 加载分组清单
def load_subjects(file_path):
    subjects = []
    with open(file_path, 'r') as fin:
        for line in fin.readlines():
            line = line.rstrip()
            subjects += [line]
    return subjects

In [19]:
#数据一致性验证
params_group_map = {}
for group_index,group_name in enumerate(data_group.keys()):
    print('{0} {1} {2} {0}'.format('-'*16, group_index, group_name))
    params_group_map[group_name] = {}
    
    subject_info_path = data_group[group_name]    
    image_root = image_root_map[group_name]
    gt_mask_root = ground_truth_mask_root_map[group_name]
    seg_prob_root = segmentation_prob_root_map[group_name]
    
    subject_list = load_subjects(subject_info_path)
    print('提取分组记录项：', len(subject_list))
    for subject in subject_list:
        image_path = os.path.join(image_root, '{}.nii.gz'.format(subject))
        gt_mask_path = os.path.join(gt_mask_root, '{}_mask.nii.gz'.format(subject)) # 实例化且转换标记的mask
        seg_prob_path = os.path.join(seg_prob_root, '{}_prob.nii.gz'.format(subject))
        
        assert os.path.exists(image_path), 'not found image in: {}'.format(image_path)
        assert os.path.exists(gt_mask_path), 'not found gt mask in : {}'.format(gt_mask_path)  
        assert os.path.exists(seg_prob_path), 'not found seg prob matrix in : {}'.format(seg_prob_path)
        param = [subject, image_path, gt_mask_path, seg_prob_path]
        params_group_map[group_name][subject] = param
        
#汇总：
print('='*32)
print('有效提取结果：')
print('\t'.join(['组名', '数量']))
for group_name, group_params in params_group_map.items():
    print('\t'.join([group_name, str(len(group_params))]))

---------------- 0 train ----------------
提取分组记录项： 1
---------------- 1 valid ----------------
提取分组记录项： 1
---------------- 2 test ----------------
提取分组记录项： 1
有效提取结果：
组名	数量
train	1
valid	1
test	1


In [20]:
#C1.1验证数据尺度一致&是否病灶分裂（不连续）
def check_shape_consistance(param):
    subject, image_path, gt_mask_path, seg_prob_path = param
    nii_img = nib.load(image_path)
    nii_gt = nib.load(gt_mask_path)
    nii_prob = nib.load(seg_prob_path)
    shape_list = [nii_img.shape, nii_gt.shape, nii_prob.shape]
    if shape_list[0] == shape_list[1] == shape_list[2]:
        return True, subject
    else:
        print('尺寸不一致：{}'.format(shape_list))
        return False, subject

In [21]:
# 加载病灶属性信息
lesion_info_map = {}

In [22]:
#C1.2验证数据尺度一致&是否病灶分裂（不连续）
def check_gt_inst_mask_lesion_connectd(param, lesion_info):
    subject, image_path, gt_mask_path, seg_prob_path = param
    nii_gt = nib.load(gt_mask_path)
    inst_mask = nii_gt.get_data()
    inst_props = measure.regionprops(inst_mask)
    assert len(inst_props) == len(lesion_info[subject]), '病灶记录数不一致：{} vs {} in {}'.format(\
                                                          len(inst_props), len(lesion_info[subject]), subject)
    check_lesion_split_status = False
    for _prop in inst_props:
        inst_label = _prop.label
        lesion_mask = _prop.image.astype(np.uint8)
        lesion_mask_re_inst_mask = measure.label(lesion_mask, connectivity=2) # 2:3D图像 8联通
        lesion_mask_re_inst_props = measure.regionprops(lesion_mask_re_inst_mask)
        if len(lesion_mask_re_inst_props) != 1:
            lesion_split_info = [(_.label, _.area) for _ in lesion_mask_re_inst_props]
            print('病灶分裂：', inst_label, '分裂信息：', lesion_split_info, subject)
            check_lesion_split_status = True
    return check_lesion_split_status, subject

In [23]:
def adjust_ww_wl(image, ww=800, wc=300, is_uint8=True):
    """
    adjust window width and window center to get proper input
    """
    min_hu = wc - (ww / 2)
    max_hu = wc + (ww / 2)
    new_image = np.clip(image, min_hu, max_hu)  # np.copy(image)
    if is_uint8:
        new_image -= min_hu
        new_image = np.array(new_image / ww * 255., dtype=np.uint8)
    return new_image

def get_auto_size(bbox_size, ctx_scale=3., min_size=36, max_size=96):
    baseline = max(bbox_size)
    # target_size = max(min_size, int(baseline * ctx_scale))
    target_size = min(max_size, max(min_size, int(baseline * ctx_scale)))
    return target_size

In [24]:
def crop_one_patch_from_image(info, img_xyz, gen_desc=True):
    # 病灶-层 裁减范围信息
    recall_lesion_label, lesion_label, lesion_slice_index, \
    w,h,target_size, cx, cy, cz, src_desc, cls_desc, slice_prob = info
    
    #确定裁减范围
    sx = int(round(cx - target_size / 2))
    sy = int(round(cy - target_size / 2))
    sz = int(round(cz - target_size / 2))
    ex = int(sx + target_size)
    ey = int(sy + target_size)
    ez = int(sz + target_size)
    # 填充矩阵偏移: 左边界超过范围
    offset_sx = 0 if sx >= 0 else - sx
    offset_sy = 0 if sy >= 0 else - sy
    offset_sz = 0 if sz >= 0 else - sz
    image_crop_patch = np.zeros((target_size,) * 3, dtype=np.uint8)

    crop_image = img_xyz[max(0, sx): ex, max(0, sy): ey, max(0, sz): ez]
    c_w, c_h, c_d = crop_image.shape  # actually crop range
    image_crop_patch[offset_sx:, offset_sy:, offset_sz:][:c_w, :c_h, :c_d] = crop_image[...]
    
    #3D转2D保存： 通道调整 xyz=>zyx
    image_crop_patch_transpose = np.transpose(image_crop_patch, (2, 1, 0))  # w,h,d => d, h, w
    d, h, w = image_crop_patch_transpose.shape # d,w,h=target_size
    image_crop_patch_transpose_2d = image_crop_patch_transpose.reshape(d * h, w)
    
    desc_info = 'x{0}_y{1}_z{2}_w{3}_h{3}_d{3}_s{4:.2f}_{5}_{6}_{7}_{8}'.format(
        int(cx), int(cy), int(cz), target_size,
        slice_prob, src_desc, cls_desc,
        lesion_label, recall_lesion_label, # 对应的GT实例化病灶(对于检出是召回，对于GT是自己) #,病灶记录,层信息有些重复
    )
    
    
    img_crop_3d_xyz, img_crop_2d_zy_x = image_crop_patch, image_crop_patch_transpose_2d
    if gen_desc:
        return img_crop_3d_xyz, img_crop_2d_zy_x, desc_info
    else:
        return img_crop_3d_xyz, img_crop_2d_zy_x
   
# test
# crop_one_patch_from_image(info=lesion_slice_info_list[0], img_xyz=img_norm)

In [25]:
def interp_image(img_xyz, new_shape_xyz):
    dtype = img_xyz.dtype
    img_xyz_5d = torch.from_numpy(img_xyz[None, None].astype(np.float32))
    img_xyz_3d = F.interpolate(img_xyz_5d, new_shape_xyz, mode='trilinear', align_corners=False).data[0, 0].numpy().astype(dtype)
    #3D转2D保存： 通道调整 xyz=>zyx
    img_transpose = np.transpose(img_xyz_3d, (2, 1, 0))  # w,h,d => d, h, w
    d, h, w = img_transpose.shape
    img_zy_x_2d = img_transpose.reshape(d * h, w)
    
    return img_xyz_3d, img_zy_x_2d

In [26]:
def get_gt_lesion_slice_info(inst_mask, inst_props):
    lesion_slice_info_list = []
    lesion_slice_mask_voxel_threshold = 3
    for _prop in inst_props:
        lesion_label = _prop.label
        x0,y0,z0,x1,y1,z1 = _prop.bbox
        lesion_mask = (inst_mask==lesion_label).astype(np.uint8)
        # 逐层计算大小
        for lesion_slice_index in range(z0, z1):
            lesion_slice_mask = lesion_mask[..., lesion_slice_index].copy()
            slice_voxel_count = (lesion_slice_mask != 0).sum()
            if slice_voxel_count < lesion_slice_mask_voxel_threshold: # 太小,跳过此层面,要求至少3个体素
                print('此层面病灶面积太小，跳过[GT]：', lesion_label, lesion_slice_index, slice_voxel_count, subject)
                continue
            lesion_inst_prop = measure.regionprops(lesion_slice_mask)
            lesion_used_prop = lesion_inst_prop[0]
            _lx0,_ly0,_lx1,_ly1 = lesion_used_prop.bbox
            w, h = _lx1 - _lx0, _ly1 - _ly0
            target_size = get_auto_size((w, h), ctx_scale=used_ctx_scale)
            cx, cy = (_lx0 + _lx1) / 2, (_ly0 + _ly1) / 2
            cz = lesion_slice_index  
            src_desc = 'gt'
            cls_desc = 'pos'
            slice_prob = 1.0
            gt_recall_lesion_label = lesion_label
            slice_lesion_bbox_param = [
                gt_recall_lesion_label, lesion_label, lesion_slice_index, 
                w,h,target_size, cx, cy, cz, 
                src_desc, cls_desc, slice_prob
            ]
            #print(slice_lesion_bbox_param)
            lesion_slice_info_list += [slice_lesion_bbox_param]

    #     break
    #pprint(lesion_slice_info_list)
    return lesion_slice_info_list

In [27]:
# 2.提取SEG层病灶信息
# 还有其他方式计算召回
def get_seg_lesion_slice_info(gt_inst_mask, prob_data, prob_threshold, subject, base_threshold=0.1, verbose=0):
    prob_data = (prob_data / 10000.).astype(np.float32) # [0-10000] => [0-1]
    #bin_mask = (prob_data >= prob_threshold).astype(np.uint8)
    #转换
    #prob_data[prob_data<base_threshold] = 0 # 清理一下概率
    bin_mask = (prob_data >= base_threshold).astype(np.uint8)    
    inst_mask = measure.label(bin_mask, connectivity=2)  # 1: 四联通, 2:八连通
    inst_props = measure.regionprops(inst_mask)
    # 保留范围内, 最大概率低于指定阈值
    pass_lesion_index_list = []
    for _prop in inst_props:
        max_prob = np.max(prob_data[inst_mask == _prop.label])
        if max_prob < prob_threshold: # 无效病灶清除
            bin_mask[inst_mask == _prop.label] = 0
            inst_mask[inst_mask == _prop.label] = 0
            pass_lesion_index_list += [_prop.label]
    
    # 重新计算
    #inst_props = measure.regionprops(inst_mask)
    inst_props = [_ for _ in inst_props if _.label not in pass_lesion_index_list]
    # print(len(inst_props))
    # 清理无效的病灶, 并构建映射信息
    seg_inst_lesion_recall_map = {}
    for _prop in inst_props:
        #print(_prop.label, _prop.area, _prop.bbox)
        x0, y0, z0, x1, y1, z1 = _prop.bbox
        w, h, d = x1 - x0, y1 - y0, z1 - z0
        if _prop.area < 10 or any(_ < 2 for _ in (w, h, d)):
            # 无效病灶, 清除
            if verbose: print('无效病灶, 清除', _prop.label, _prop.area, _prop.bbox, subject)
            bin_mask[inst_mask == _prop.label] = 0
            inst_mask[inst_mask == _prop.label] = 0
            continue
        # pass big lesion
        elif _prop.area >= 20000:
            output = '异常超巨大病灶, 注意: {} {} {} {}'.format(_prop.label, _prop.area, _prop.bbox, subject)
            print("\033[31m{}\033[0m".format(output))
#             bin_mask[inst_mask == _prop.label] = 0
#             inst_mask[inst_mask == _prop.label] = 0
            
        #建立GT病灶召回映射
        seg_inst_label = _prop.label
        match_gt_lesion_list = np.unique(gt_inst_mask[inst_mask == _prop.label]).tolist()
        match_gt_lesion_list = sorted([str(_) for _ in match_gt_lesion_list if 0 != _]) # int => str
        match_gt_lesion_count = len(match_gt_lesion_list)
        if match_gt_lesion_count == 0:
            seg_inst_lesion_recall_map[seg_inst_label] = '0' # 0
        elif match_gt_lesion_count == 1:
            seg_inst_lesion_recall_map[seg_inst_label] = match_gt_lesion_list[0]
        else:
            print('\033[34m[warning]match more than one, use small/all?:\033[0m', _prop.area, seg_inst_label, '=>', match_gt_lesion_list, subject)
            #排除此种
            # 仅保留最小的
            #seg_inst_lesion_recall_map[seg_inst_label] = match_gt_lesion_list[0]
            # 建立一对多            
            seg_inst_lesion_recall_map[seg_inst_label] = "&".join(match_gt_lesion_list)

    #print(seg_inst_lesion_recall_map)

    lesion_slice_info_list = []
    lesion_slice_mask_voxel_threshold = 3
    for _prop in inst_props:
        seg_inst_label = _prop.label
        if seg_inst_label not in seg_inst_lesion_recall_map:
            if verbose: print('病灶排除，跳过[SEG]：', seg_inst_label)
            continue
        recall_lesion_label = seg_inst_lesion_recall_map[seg_inst_label]
        #print(seg_inst_label, recall_lesion_label)
        x0,y0,z0,x1,y1,z1 = _prop.bbox
        lesion_mask = (inst_mask==seg_inst_label).astype(np.uint8)
        # 逐层计算大小
        for lesion_slice_index in range(z0, z1):
            lesion_slice_mask = lesion_mask[..., lesion_slice_index].copy()
            slice_voxel_count = (lesion_slice_mask != 0).sum()
            if slice_voxel_count < lesion_slice_mask_voxel_threshold: # 太小,跳过此层面,要求至少3个体素
                if verbose: print('此层面病灶面积太小，跳过[SEG]：', seg_inst_label, lesion_slice_index, slice_voxel_count, subject)
                continue
            lesion_inst_prop = measure.regionprops(lesion_slice_mask)
            lesion_used_prop = lesion_inst_prop[0]
            _lx0,_ly0,_lx1,_ly1 = lesion_used_prop.bbox
            w, h = _lx1 - _lx0, _ly1 - _ly0
            target_size = get_auto_size((w, h), ctx_scale=used_ctx_scale)
            cx, cy = (_lx0 + _lx1) / 2, (_ly0 + _ly1) / 2
            cz = lesion_slice_index  
            src_desc = 'seg'
            cls_desc = ['pos','neg'][recall_lesion_label=='0'] # int => str
            slice_prob = np.max(prob_data[...,lesion_slice_index][lesion_slice_mask!=0])
            slice_lesion_bbox_param = [
                recall_lesion_label, seg_inst_label, lesion_slice_index, 
                w,h,target_size, cx, cy, cz, 
                src_desc, cls_desc, slice_prob
            ]
            #print(slice_lesion_bbox_param)
            lesion_slice_info_list += [slice_lesion_bbox_param]

    #     break
    #pprint(lesion_slice_info_list)
    return lesion_slice_info_list, inst_mask
    

# seg_lesion_slice_info_list = get_gt_lesion_slice_info(inst_mask, inst_props)

# lesion_slice_info_list = gt_lesion_slice_info_list #+seg_lesion_slice_info_list

In [28]:
def process_one_group_data(group_name, param):
    #数据处理
    #gt/seg 动脉瘤类别标注不同，分别处理
    subject, image_path, gt_mask_path, seg_prob_path = param

    nii_mask_gt = nib.load(gt_mask_path)
    inst_mask = np.asarray(nii_mask_gt.dataobj)
    inst_props = measure.regionprops(inst_mask)
    # 1.提取GT层病灶信息
    gt_lesion_slice_info_list = get_gt_lesion_slice_info(inst_mask, inst_props)
    gt_inst_mask = inst_mask # SEG计算匹配召回使用!

    #2.提取SEG的Prob信息
    prob_threshold = 0.3
    nii_seg_prob = nib.load(seg_prob_path)
    seg_prob = np.asarray(nii_seg_prob.dataobj)
    seg_lesion_slice_info_list, seg_inst_mask_filter = get_seg_lesion_slice_info(gt_inst_mask, seg_prob, prob_threshold, subject)
    seg_save_root = os.path.join(save_root, 'prob2seg_with_filter', group_name)
    seg_name = '{}_seg.nii.gz'.format(subject)
    os.makedirs(seg_save_root, exist_ok=True)
    seg_save_full_path = os.path.join(seg_save_root, seg_name)
    nii_seg = nib.Nifti1Image(seg_inst_mask_filter.astype(np.uint8), nii_seg_prob.affine)
    nii_seg.to_filename(seg_save_full_path)
    
    
    # 合并病灶信息
    lesion_slice_info_list = gt_lesion_slice_info_list + seg_lesion_slice_info_list

    # 图像加载&归一化
    nii_img = nib.load(image_path)
    img_ori = np.asanyarray(nii_img.dataobj)
    WW, WL = ww_wl
    img_norm = adjust_ww_wl(img_ori, ww=WW, wc=WL, is_uint8=True) # 裁减&归一化，uint8,[0,255]


    # 开始采样
    group_data_info_list = []
    for slice_info in lesion_slice_info_list:
        img_crop_3d_xyz, img_crop_2d_zy_x, desc_info = crop_one_patch_from_image(info=slice_info, img_xyz=img_norm)
        #shape_origin_save_dir = '{}/shape_origin/{}/{}'.format(save_root, group_name, subject)

        #插值,归一化大小
        #new_shape = (32, 32, 32)
        img_xyz_3d, img_zy_x_2d = interp_image(img_xyz=img_crop_3d_xyz, new_shape_xyz=shape_norm)
        shape_norm_save_dir = '{}/shape_norm/{}/{}'.format(save_root, group_name, subject)
        name = '{}.png'.format(desc_info)
        save_full_path = os.path.join(shape_norm_save_dir, name)
        os.makedirs(shape_norm_save_dir, exist_ok=True)
        cv2.imwrite(save_full_path, img_zy_x_2d)

        sub_path = '{}/{}'.format(subject, name)
        cls = [1, 0][sub_path.endswith('_0.png')] # *_0.png结尾的,表示无召回，假阳
        group_data_info_list += [(sub_path, cls)]

    return group_data_info_list

# test
# param = group_params_list[2]
# process_one_group_data(group_name, param)

In [29]:
def aneu_eval_auc_indicator(eval_patch_info_list):
    # seg lesion/slice patch props auc calc
    auc_indicator_base_slice = []
    auc_indicator_base_lesion = {}
    
    
    all_slice_patch_count = {'pos': 0, 'neg': 0, 'all': 0}
    seg_slice_patch_count = {'pos': 0, 'neg': 0, 'all': 0}
    seg_lesion_count = {'pos': set(), 'neg': set(), 'all': set()}
    case_count = {'pos': set(), 'neg': set(), 'all': set()}
    all_lesion_count = {'seg': set(), 'gt': set(), 'all': set()}
    match_gt_lesion_count = set()
    gt_sample_lesion_count = set()
    
    
    # stage1 slice patch and lesion match gt info
    for sub_path in eval_patch_info_list:
        #eg: x151_y127_z172_w32_h32_d32_s0.39_seg_neg_5_0.png
        series_name, slice_patch_info = sub_path.split('/')
        *slice_info, seg_gt_label, pos_neg_label, inst_index, recall_inst_index = slice_patch_info.split('_')
        
        # all
        slice_patch_index = sub_path
        lesion_index = f'{series_name};{inst_index}'
        
        all_slice_patch_count['all'] += 1  # 全部层面: GT+SEG
        case_count['all'].add(series_name) # 全部病例: 阴性+阳性
        lesion_key = f'{series_name};{seg_gt_label};{inst_index}'
        all_lesion_count['all'].add(lesion_key)
        
        if 'gt' == seg_gt_label:
            case_count['pos'].add(series_name) # 阳性病例
            all_slice_patch_count['pos'] += 1            
            
            gt_lesion_key = f'{series_name};{inst_index}' # var:inst_index same to var:recall_inst_index
            gt_sample_lesion_count.add(gt_lesion_key) # 阳性病灶(GT)
            all_lesion_count['gt'].add(lesion_key)            
            
            # continue
        elif 'seg' == seg_gt_label:
            # count
            seg_slice_patch_count['all'] += 1  #
            seg_slice_patch_count[pos_neg_label] += 1
            all_slice_patch_count[pos_neg_label] += 1
            
            seg_lesion_key = f'{series_name};{inst_index}'
            seg_lesion_count['all'].add(seg_lesion_key)
            seg_lesion_count[pos_neg_label].add(seg_lesion_key)
            all_lesion_count['seg'].add(lesion_key)
            
            if 'pos' == pos_neg_label:
                recall_inst_index_list = recall_inst_index.rsplit('.png', maxsplit=1)[0]
                recall_inst_index_list = recall_inst_index_list.split('&')
                for recall_inst_index in recall_inst_index_list:
                    match_gt_lesion_key = f'{series_name};{recall_inst_index}'
                    match_gt_lesion_count.add(match_gt_lesion_key)
            
            # auc
            s1_score = float(slice_info[-1][1:])
            # 1. slice patch level of seg props only
            #slice_type = int(_cls)
            slice_type = int('pos' == pos_neg_label)
            auc_indicator_base_slice += [(slice_patch_index, s1_score, slice_type)]
            
            # 2. lesion level of seg props only            
            #lesion_recall_type = int(_cls)
            lesion_recall_type = int('pos' == pos_neg_label)
            # 2.1 lesion in stage1
            auc_indicator_base_lesion.setdefault(lesion_index, [-1, lesion_recall_type])
            cur_lesion_score_s1 = auc_indicator_base_lesion[lesion_index][0]
            auc_indicator_base_lesion[lesion_index][0] = max(cur_lesion_score_s1, s1_score)      
            
        else:
            raise Exception('unknown origin type: {} in {}'.format(seg_gt_label, sub_path))
    
    case_count['neg'] = case_count['all'] - case_count['pos']
    print('case count:', {k:len(v) for k, v in case_count.items()})
    print('all lesion count:', {k:len(v) for k, v in all_lesion_count.items()})
    seg_lesion_count_map = {k:len(v) for k, v in seg_lesion_count.items()}
    print('seg lesion count:', seg_lesion_count_map, ',and max match gt lesion:', len(match_gt_lesion_count))
    
    
    # slice count 
    print('seg slice count:', seg_slice_patch_count)
    print('all slice count:', all_slice_patch_count)
    
    #ipdb.set_trace()
    seg_auc_info_list = []
    group_name_list= ['seg-slice-auc', 'seg-lesion-auc']
    for group_name, seg_slice_info in zip(group_name_list, [auc_indicator_base_slice, [[k]+v for k, v in auc_indicator_base_lesion.items()]]):
        pred_score_list = [_[1] for _ in seg_slice_info]
        label_list = [_[2] for _ in seg_slice_info]
        all_neg = all([0 == _ for _ in label_list])
        all_pos = all([1 == _ for _ in label_list])
        if all_neg or all_pos:
            auc_score = -1.00
        else:
            y_pred_list = np.array(pred_score_list)
            y_gt_list = np.array(label_list)
            auc_score = roc_auc_score(y_true=y_gt_list, y_score=y_pred_list)
        info = f'{group_name}={auc_score:.4f}'
        seg_auc_info_list += [info]
        
    print('AUC', seg_auc_info_list)

In [30]:
for group_name, group_params in params_group_map.items():
    if group_name in [
        # 'train',
        # 'valid',
        # 'test',
    ]:
        continue
    print('{0} {1} {0}'.format('='*16, group_name))
    group_params_list = sorted(group_params.values())
    results = Parallel(n_jobs=cpu_count()//4, verbose=5, backend='multiprocessing')(\
            delayed(process_one_group_data)(group_name, _) for _ in group_params_list[:])
    #
    save_name = os.path.join(save_root, 'part_{}.txt'.format(group_name))
    info_list = []
    with open(save_name, 'w') as fout:
        for case_info in results:
            for slice_info in case_info:
                desc, _cls = slice_info
                info_list += ['{}\t{}'.format(desc, _cls)]
                #print(desc, _cls, file=fout)
        print('数据总条目:', len(info_list))
        fout.write('\n'.join(info_list))
        # 分割病灶
        data_info = [_.split()[0] for _ in info_list]
        aneu_eval_auc_indicator(eval_patch_info_list=data_info)
    shape_norm_save_dir = '{}/shape_norm/{}'.format(save_root, group_name)
    shutil.copy(save_name, shape_norm_save_dir)
#     break

================ train ================


[Parallel(n_jobs=6)]: Using backend MultiprocessingBackend with 6 concurrent workers.


数据总条目: 39
case count: {'pos': 1, 'neg': 0, 'all': 1}
all lesion count: {'seg': 4, 'gt': 1, 'all': 5}
seg lesion count: {'pos': 1, 'neg': 3, 'all': 4} ,and max match gt lesion: 1
seg slice count: {'pos': 12, 'neg': 15, 'all': 27}
all slice count: {'pos': 24, 'neg': 15, 'all': 39}
AUC ['seg-slice-auc=0.7972', 'seg-lesion-auc=0.8333']
================ valid ================


[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    7.3s finished
[Parallel(n_jobs=6)]: Using backend MultiprocessingBackend with 6 concurrent workers.


数据总条目: 39
case count: {'pos': 1, 'neg': 0, 'all': 1}
all lesion count: {'seg': 4, 'gt': 1, 'all': 5}
seg lesion count: {'pos': 1, 'neg': 3, 'all': 4} ,and max match gt lesion: 1
seg slice count: {'pos': 12, 'neg': 15, 'all': 27}
all slice count: {'pos': 24, 'neg': 15, 'all': 39}
AUC ['seg-slice-auc=0.7972', 'seg-lesion-auc=0.8333']
================ test ================


[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    7.5s finished
[Parallel(n_jobs=6)]: Using backend MultiprocessingBackend with 6 concurrent workers.


数据总条目: 39
case count: {'pos': 1, 'neg': 0, 'all': 1}
all lesion count: {'seg': 4, 'gt': 1, 'all': 5}
seg lesion count: {'pos': 1, 'neg': 3, 'all': 4} ,and max match gt lesion: 1
seg slice count: {'pos': 12, 'neg': 15, 'all': 27}
all slice count: {'pos': 24, 'neg': 15, 'all': 39}
AUC ['seg-slice-auc=0.7972', 'seg-lesion-auc=0.8333']


[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:    6.8s finished
